<a href="https://colab.research.google.com/github/Facusc16/ufc_web_scraper/blob/main/ufc_webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requerimientos

## Librerías

In [ ]:
import pandas as pd
from google.colab import drive
import requests
from bs4 import BeautifulSoup
import json
from IPython.display import clear_output
import time
from datetime import date

## Conectar con Google Drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Definir Scrapers

## Setup de request: headers, cookies, y params

In [ ]:
# URL para solicitar los datos de todos los peleadores vía AJAX
url = "https://www.ufcespanol.com/views/ajax"

# Encabezados HTTP para simular una solicitud del navegador
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 OPR/120.0.0.0',
    'Referer': 'https://www.ufcespanol.com/',
    'Origin': 'https://www.ufcespanol.com',
    'X-Requested_With': 'XMLHttpRequest',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Languege': 'es-ES,es;q=0.9,en;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br, zstd'
}

# Cookies para mantener la sesión y evitar bloqueos del servidor
cookies= {
    'OptanonAlertBoxClosed': '2024-12-08T03:18:48.604Z',
    'AMP_8a1a981826': 'JTdCJTIyZGV2aWNlSWQlMjIlM0ElMjIwZDY4YTdjMy03YzE2LTQ0ODEtYjMwNS1hZDRhYWQ1YzYwOTYlMjIlMkMlMjJzZXNzaW9uSWQlMjIlM0ExNzUyODE0MzgxOTU0JTJDJTIyb3B0T3V0JTIyJTNBZmFsc2UlMkMlMjJsYXN0RXZlbnRUaW1lJTIyJTNBMTc1MjgxNDM5MDI4NiUyQyUyMmxhc3RFdmVudElkJTIyJTNBMTIlN0Q=',
    '__gads': 'ID=eafed2d9b0505890:T=1733627707:RT=1753466879:S=ALNI_MYfO1z43UKkl-A1_eYbALFN3v6Srg',
    '__gpi': 'UID=00000f7dba2c1e77:T=1733627707:RT=1753466879:S=ALNI_MYf7tCMYB14gYoxvLF9Z-B-lPvFkw',
    '__eoi': 'ID=402899fb4de80cff:T=1749228520:RT=1753466879:S=AA-AfjY6n-z37XY2U9F8Vbv2uMx7',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Fri+Jul+25+2025+15%3A10%3A01+GMT-0300+(hora+est%C3%A1ndar+de+Argentina)&version=202411.2.0&browserGpcFlag=0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=1%3A1%2C2%3A0%2C3%3A0%2C4%3A0%2C5%3A0&geolocation=%3B&AwaitingReconsent=false&consentId=8fced91d-35c4-4dfe-9f50-94d68c44bf1c&interactionCount=0&isAnonUser=1'
}

# Parámetros requeridos por el backend de la UFC para devolver los peleadores
params = {
    'view_name': 'all_athletes',
    'view_display_id': 'page',
    'view_args': '',
    'view_path': '/athletes/all',
    'view_base_path': '',
    'view_dom_id': 'f3c4fa5f532ca353fe06e7bd7f1bbd7fbc6efaa8be071bba1459dd35271cc588',
    'pager_element': 0,
    'page': 0,
    'ajax_page_state[theme]': 'ufc',
    'ajax_page_state[theme_token]': '',
    'ajax_page_state[libraries]': 'eJx1UkGOwyAM_FAIT0IOOIlbgiPstE1fvwSodg9bCaGZwRjbA4SgDOm00ME4Z046TODvTrms3f7B7iZfjxRf5R6qYnb42lkwuJlioWLhKEFyTBt9DVkwYYY4hAfaBwXkYQaPKjbkY4c4NjZGSnfzpLCg_h_wIHyKgRu8hggnH-oCiecH5tNyQs9x2DnGz60LmyupDHKK4lZ6EhyO2dvpUOUkFbcqjWBEr1VZIk8QjRf5S2-N1YQmZN4DP1OXRCktBnSNqO2FDdNhNqAWscPSZCntT5DNTFm0K4pmZi41_PIVIXT-xEkzpiCGd6WN3jWRgyA2lxlPkFKLdH1mgpD9Wgapa-n-sqUe0rLqDlINWznTG5S4Vucie4hdKEmXj94y2V84Qoyud1mn4c7yM_qLpZQ8VIts3ceNwxGxSe5yzRVn7Qd0ndJMqbTsxOfLu-bxRzVN_QFZ8BWy'
}

## Funciones

In [ ]:
def data_filler(soup, class_):
  tag = soup.find(class_=class_)
  return tag.text.strip() if tag else None

In [ ]:
def safe_int(soup, id):
  el = soup.find(id=id)
  return int(el.text) if el and el.text.isdigit() else None

In [ ]:
def get_tags(soup, keys, div, label_class, value_class):
  data = {key: None for key in keys}

  values = [value for value in soup.find_all(class_=div)]

  for value in values:
    label_tag = value.find(class_=label_class).text.strip().lower().replace(".", "").replace(" ", "_")
    value_tag = value.find(class_=value_class).text.split("(")[0].strip()

    try:
      if value_tag == "":
        value_tag = None
      elif "." in value_tag:
        value_tag = float(value_tag)
      else:
        value_tag = int(value_tag)
    except ValueError:
      pass

    data[label_tag] = value_tag

  return data

## Profiles Scraper

In [ ]:
def get_fighter_profile(url):

  try:
    html = requests.get(url, headers=headers)
    soup = BeautifulSoup(html.content, 'html.parser')

    method_position = get_tags(soup, ['ko/tko', 'dec', 'sub', 'standing', 'clinch', 'ground'], "c-stat-3bar__group", "c-stat-3bar__label", "c-stat-3bar__value")
    strikes_takedowns = get_tags(soup, ["sig_strikes_landed", "sig_strikes_attempted", "takedowns_landed", "takedowns_attempted"], "c-overlap__stats", "c-overlap__stats-text", "c-overlap__stats-value")
    bio = get_tags(soup, ['place_of_birth', 'fighting_style', 'age', 'height', 'weight', 'octagon_debut','reach', 'leg_reach'], "c-bio__field", "c-bio__label", "c-bio__text")

    fighter_profile = {
      'name': data_filler(soup, "hero-profile__name"),
      'nickname': data_filler(soup, "hero-profile__nickname"),
      'place_of_birth': bio['place_of_birth'],
      'age': bio['age'],
      'height': bio['height'],
      'weight': bio['weight'],
      'reach': bio['reach'],
      'leg_reach': bio['leg_reach'],
      'fighting_style': bio['fighting_style'],
      'ufc_debut_date': bio['octagon_debut'],
      'weigh_class': data_filler(soup, "hero-profile__division-title"),
      'professional_record': data_filler(soup, "hero-profile__division-body"),
      'ko_tko_wins': method_position['ko/tko'],
      'decision_wins': method_position['dec'],
      'submission_wins': method_position['sub'],
      'significant_strikes_landed': strikes_takedowns['sig_strikes_landed'],
      'significant_strikes_attempted': strikes_takedowns['sig_strikes_attempted'],
      'strikes_by_standing': method_position['standing'],
      'strikes_by_clinch': method_position['clinch'],
      'strikes_by_ground': method_position['ground'],
      'strikes_to_the_head': safe_int(soup, 'e-stat-body_x5F__x5F_head_value'),
      'strikes_to_the_body': safe_int(soup, 'e-stat-body_x5F__x5F_body_value'),
      'strikes_to_the_leg': safe_int(soup, 'e-stat-body_x5F__x5F_leg_value'),
      'takedowns_landed': strikes_takedowns['takedowns_landed'],
      'takedowns_attempted': strikes_takedowns['takedowns_attempted'],
      'is_active': True if soup.find(class_="hero-profile__tag") else False
    }
    return fighter_profile

  except AttributeError:
    return None

## Fighters Scraper

In [ ]:
def ufc_fighters_scraper(url, headers, cookies, params):
  fighters_profiles_list = []
  count = 0

  while True:

    response = requests.get(url, headers=headers, cookies=cookies, params=params)
    data = response.json()

    html = None
    for item in data:
      if item['command'] == 'insert':
        html = item['data']
        break

    soup = BeautifulSoup(html, 'html.parser')
    href_in_page = soup.find_all('a', class_='e-button--black')

    if not href_in_page:
      break

    for a_tag in href_in_page:
      fighter_profile = get_fighter_profile('https://www.ufc.com' + a_tag['href'])
      if fighter_profile is not None:
        fighters_profiles_list.append(fighter_profile)
      count +=1

    print(f"Registrados {len(fighters_profiles_list)} peleadores")
    clear_output(wait=True)

    params['page'] += 1
    time.sleep(1)

  return pd.DataFrame(fighters_profiles_list)

# Crear csv

In [ ]:
today_date = date.today().strftime("%d_%m_%Y")
file_name = f'ufc_fighters_profiles_{today_date}.csv'
path = f'/content/drive/MyDrive/ufc_scraper/{file_name}'

try:
  fighters_profiles = pd.read_csv(path)
  print("Cargado desde archivo")

except FileNotFoundError:
  fighters_profiles = ufc_fighters_scraper(url, headers=headers, cookies=cookies, params=params)
  fighters_profiles.to_csv(path, index=False)
  print("Guardado en archivo")

Guardado en archivo


In [ ]:
fighters_profiles

,name,nickname,place_of_birth,age,height,weight,reach,leg_reach,fighting_style,ufc_debut_date,...,significant_strikes_attempted,strikes_by_standing,strikes_by_clinch,strikes_by_ground,strikes_to_the_head,strikes_to_the_body,strikes_to_the_leg,takedowns_landed,takedowns_attempted,is_active
0,Danny Abbadi,"""The Assassin""","Orlando, United States",39.0,71.0,156.0,NaN,NaN,None,"Jun. 24, 2006",...,155.0,42.0,14.0,3.0,51.0,None,0.0,NaN,NaN,True
1,Nariman Abbassov,None,Kazakhstan,29.0,NaN,156.0,NaN,NaN,None,"Aug. 9, 2025",...,225.0,45.0,0.0,0.0,26.0,None,11.0,NaN,2.0,True
2,Tank Abbott,"""Tank""","Huntington Beach, United States",NaN,72.0,253.0,NaN,NaN,None,"Jul. 14, 1995",...,63.0,5.0,11.0,3.0,17.0,None,0.0,NaN,3.0,True
3,Hamdy Abdelwahab,"""The Hammer""",Egypt,32.0,74.0,264.5,72.0,41.0,Wrestler,"Jul. 30, 2022",...,320.0,125.0,2.0,30.0,110.0,None,29.0,3.0,6.0,True
4,Mansur Abdul-Malik,None,"Pittsburgh, United States",27.0,74.0,186.0,79.5,43.0,Freestyle,"Nov. 9, 2024",...,268.0,52.0,14.0,65.0,119.0,None,6.0,NaN,3.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3021,Farès Ziam,"""Smile Killer""","Vénissieux, Auvergne-Rhône-Alpes, France",27.0,73.0,156.0,75.0,42.0,Kickboxer,"Sep. 7, 2019",...,661.0,241.0,42.0,59.0,223.0,None,52.0,NaN,32.0,True
3022,James Zikic,"""The Messiah""","Watford, United Kingdom",46.0,74.0,202.0,NaN,NaN,None,"Jul. 13, 2002",...,125.0,18.0,16.0,10.0,24.0,None,9.0,NaN,4.0,True
3023,Cat Zingano,"""Alpha""","Winona, United States",40.0,66.0,144.0,68.0,38.0,None,"Apr. 14, 2013",...,295.0,66.0,44.0,71.0,107.0,None,28.0,13.0,20.0,True
3024,Igor Zinoviev,None,United States,NaN,0.0,0.0,NaN,NaN,None,"Mar. 13, 1998",...,NaN,0.0,0.0,0.0,0.0,None,0.0,NaN,NaN,True
